### Import libraries

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Resizing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD

import matplotlib.pyplot as plt
import numpy as np

from cifar10_classes import class_map

### Load and assess the Cifar10 dataset

In [ ]:
def load_and_assess_dataset():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    
    print(f'X_train.shape: {X_train.shape}')
    print(f'X_test.shape: {X_test.shape}')
    print(f'y_train.shape: {y_train.shape}')
    print(f'y_test.shape: {y_test.shape}')
    
    k = np.random.randint(0, X_train.shape[0])
    plt.title(f'Image #{k} (label: "{class_map[y_train[k][0]]}")')
    plt.imshow(X_train[k])

    return (X_train, y_train), (X_test, y_test)

### Preprocess the train and test data

In [ ]:
def preprocess_data(X_train, y_train, X_test, y_test):
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    
    y_train = to_categorical(y_train, dtype='uint8')
    y_test = to_categorical(y_test, dtype='uint8')
    
    num_classes = y_train.shape[1]
    print(f'num_classes: {num_classes}')

    return X_train, y_train, X_test, y_test, num_classes

### Build model architecture and compile

In [ ]:
def build_model(num_classes):
    model = Sequential()

    model.add(Resizing(32,32))
    model.add(Conv2D(96, (5,5), input_shape=(32,32,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(80, (5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(96, (5,5), padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (5,5), padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    sgd = SGD(learning_rate=0.01, momentum=0.9, decay=(0.01/25), nesterov=False)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

### Train the model with training data

In [ ]:
def train_model(X_train, y_train, model, batch_size=32, epochs=20):
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.15)
    return model

### Evaluate the model on test data

In [ ]:
def evaluate_model(X_test, y_test, model):
    result = model.evaluate(X_test, y_test)
    print(model.metrics_names)
    print(result)

In [ ]:
def save_trained_model(model, model_name='cifar10_trained_model.h5'):
    model.save(model_name)

In [ ]:
(X_train, y_train), (X_test, y_test) = load_and_assess_dataset()

In [ ]:
X_train, y_train, X_test, y_test, num_classes = preprocess_data(X_train, y_train, X_test, y_test)

In [ ]:
model = build_model(num_classes)

In [ ]:
model = train_model(X_train, y_train, model, batch_size=128, epochs=10)

In [ ]:
model.summary()

In [ ]:
evaluate_model(X_test, y_test, model)

In [ ]:
save_trained_model(model, 'cifar10_trained_model.h5')